# Fortgeschrittene Deep-Learning-Techniken

## Nichtlineare Modelle
Oftmals ist es von Nutzen, über eine rein lineare Anordnung der Schichten hinauszugehen. Ein möglicher Fall ist ein multimodaler Inputs, bspw. willst du den Preis eines Produktes voraussage, indem eine Textbeschreibung, Verkaufszahlen und ein Bild des Produkts eingespeist werden, weshalb CNN-, RNN- und dicht vernetzte Schichten als Input-Schichten benötigt werden.
<img src="./imgs/Multimodal_Input.png">
Eine weitere Möglichkeit ist multimodaler Output. So kann gleichzeitig das Genre und das Veröffentlichungsdatum eines Buches ermittelt werden. Da wahrscheinlich Korrelation zwischen beiden Eigenschaften existiert, ist es von Nutzen, eine gemeinsame Basis von Schichten zu nutzen.
<img src="./imgs/Multimodal_Output.png">
Ein weiterer Trend ist das Einspeisen des Outputs einer früheren Schicht als Teilinput einer späteren Schicht, um Informationsverlust zu verhindern.
<img src="./imgs/nonlinear_cnn.png">

## Vergleich der Sequentiellen API mit der Funktionalen API 

In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import Input

# Zunächst sequentiellen Modell
seq_model = Sequential()
seq_model.add(Dense(32, activation = 'relu', input_shape = (64, )))
seq_model.add(Dense(32, activation = 'relu'))
seq_model.add(Dense(10, activation = 'softmax'))
print(seq_model.summary())

# Funktionales Modell
input_tensor = Input(shape = (64, )) # extra Defintion des Inputs
x = Dense(32, activation = 'relu')(input_tensor) # Sichten geben wieder Funktion, deren Parameter vorangegangene Schicht ist
x = Dense(32, activation = 'relu')(x)
output_tensor = Dense(10, activation = 'relu')(x)

model = Model(input_tensor, output_tensor) # Modell wird letztendlich bereit gemacht durch diesen Befehl
# Hier bringt Keras alle Schichten vom Input zum Output zusammen in eine azyklische Struktur
# Keras prüft, ob der Input über einen Graphen zurückführend zum Input führt
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
None
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
__________________________________

## Multi-Input-Modelle
Diese Modelle müssen ihre Inputs an einem Punkt vereinigen, was typischerweise mit
* `keras.layers.add` oder
* `keras.layers.concatenate`
Als Beispiel gilt hier ein Frage-Antwort-System. Inputs sind der Referenztext und eine Frage und der Output ist die Antwort, `softmax` über ein vordefiniertes Vokabular. 
<img src="./imgs/question_answer.png">


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input

text_vocab_size = 10000
question_vocab_size = 10000
answer_vocab_size = 500

text_input = Input(shape = (None, ), dtype = 'int32', name = 'text')
embedded_text = layers.Embedding(64, text_vocab_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape = (None, ), dtype = 'int32', name = 'question')
embedded_question = layers.Embedding(32, question_vocab_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenate = layers.concatenate([encoded_text, encoded_question], axis = -1)

answer = layers.Dense(answer_vocab_size, activation = 'softmax')(concatenate)

model = Model([text_input, question_input], answer)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
____________________________________________________________________________________________

Analog wird ein Multi-Output-Modell erstellt.

### Flaschenhälse für Repäsentation
Jede Schicht hat in einem sequentiellen Modell nur Zugriff auf die Information, die von der vorhergehenden Schicht weitergegeben wurde. Ist eine bestimmte Schicht kleiner als benötigt, so kann diese auch nur diese kleinere Menge an Informationen weitergeben. Die Schicht "schneidet" nützliche Information vorzeitig ab.

### Vanishing Gradient
Wird ein Netzwerk zu groß, kann das Feedbacksignal der Backpropagation nur geschwächt oder gar nicht in die ersten Schichten vordringen, was vanishing gradient problem genannt wird.

## Layer Weight Sharing
Eine wichtige Eigenschaft der funktionalen API ist die Möglichkeit, Schichten wiederzuverwenden. Das emöglicht es verschiedenen Modellen Zweige zu besitzen, die auf den gleichen Gewichten agieren und somit die gleichen Repäsentation.

Ein Beispiel wäre ein Modell, das die semantische Ähnlichkeit zweier Sätze beurteilen soll. Es hat die Inputs der Sätze A und B. Semantische Ähnlichkeit ist eine symmetrische Beziehung: die Beziehung zwischen A und B ist die gleiche wie zwischen B und A. Wir wollen also keine zwei seperate Modelle erstellen, sondern eines, in dem sich beide Versionen eine LSTM-Schicht teilen, ein sogenanntes *siamese LSTM model*.

In [3]:
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

lstm_shared = layers.LSTM(32) # gemeinsame Schicht

left_input = Input(shape=(None, 128)) # variable length input of size 128
left_output = lstm_shared(left_input) # sind outputs, die über lstm_shared entstehen, wenn Input linker Input ist

right_input = Input(shape=(None, 128))
right_output = lstm_shared(right_input) 

merged = layers.concatenate([left_output, right_output], axis = -1)
predictions = layers.Dense(1, activation = 'sigmoid')(merged) 

model = Model([left_input, right_input], predictions)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 32)           20608       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 64)           0           lstm_2[0][0]               